# Torch Channel Pruning Example Code

The following notebook is an example as to how one may compress their model via Channel Pruning using the AIMET library. The general procedure for compressing is to use AIMET's ModelCompressor, after specifying parameters determining the manner of compression, to compress the model, then finetuning it to recover lost accuracy.

We now present an overview of the technique. Recall that in any model, a convolutional layer is defined by four dimensions (m, n, h, w), where m and n are the number of input and output channels, respectively; and h and w are the height and width of the convolutional kernel. Channel Pruning seeks to reduce the number of input channels in this convolutional layer. There are two steps - winnowing, which removes less informative channels, and weight reconstruction, which seeks to shift the weights such that a linear regression between the old outputs and new outputs exists with minimal error.

#### The example code shows the following:
1. Instantiate Data Pipeline for evaluation
2. Load the pretrained resnet18 Pytorch model and get starting accuracy
3. Compress using channel pruning and obtain resulting accuracy
4. Finetune and obtain final accuracy

The first three cells below take care of all necessary imports:

In [ ]:
import warnings
warnings.filterwarnings("ignore", ".*param.*")

# Imports necessary for the notebook
import os
from typing import Tuple
from datetime import datetime
from decimal import Decimal
import torch
from torchvision.models import resnet18

In [ ]:
# AIMET Imports
from aimet_torch.compress import ModelCompressor
from aimet_common.defs import CompressionScheme, CostMetric
from aimet_torch.defs import GreedySelectionParameters, ChannelPruningParameters

In [ ]:
# Imports needed for the Data Pipeline
from Examples.common import image_net_config
from Examples.torch.utils.image_net_data_loader import ImageNetDataLoader
from Examples.torch.utils.image_net_evaluator import ImageNetEvaluator
from Examples.torch.utils.image_net_trainer import ImageNetTrainer

## Setting Up Our Config Dictionary

The config dictionary specifies a number of things for use in the pipeline:
* dataset_dir: Path to a directory containing ImageNet dataset. This folder should contain at least 2 subfolders: 'train': for training dataset and 'val': for validation dataset.
* use_cuda: A boolean var to indicate to run the test on GPU.
* logdir: Path to a directory for logging.
* epochs: Number of epochs to be used in fine tuning.
* learning_rate: A float type learning rate for model finetuning.
* learning_rate_schedule: A list of epoch indices for learning rate schedule used in finetuning. Check https://pytorch.org/docs/stable/_modules/torch/optim/lr_scheduler.html#MultiStepLR for more details.

The config dictionary is used for all of the remaining cells. To get a better understanding of when each of the parameters in the config dictionary is used, refer to the code in those cells.

**Note**: You will have to replace the dataset_dir path with the path to your own imagenet/tinyimagenet dataset.

In [ ]:
config = {'dataset_dir': "path/to/dataset", # Replace with the directory of your dataset!
          'use_cuda': True,
          'logdir': os.path.join("benchmark_output", "channel_pruning_"+datetime.now().strftime("%Y-%m-%d-%H-%M-%S")),
          'epochs': 15,
          'learning_rate': 1e-2,
          'learning_rate_schedule': [5, 10]
         }

os.makedirs(config['logdir'], exist_ok=True)

## 1. Instantiate Data Pipeline

The next cell defines the data pipeline. The ImageNetDataPipeline class takes care of both evaluating and finetuning a model using a dataset directory (which should contain both training data and validation data, already separated into folders) that is specified by the user. For more detail on how it works, see the relevant files under examples/torch/utils.

The data pipeline class is simply a template for the user to follow. The methods for this class can be replaced by the user to fit their needs.

In [ ]:
class ImageNetDataPipeline:
    """
    Provides APIs for model quantization using evaluation and finetuning.
    """

    def __init__(self, config):
        """
        :param config:
        """
        self._config = config

    def data_loader(self):
        """
        :return: ImageNetDataloader
        """
        
        data_loader = ImageNetDataLoader(is_training=False, images_dir=self._config["dataset_dir"],
                                         image_size=image_net_config.dataset['image_size']).data_loader

        return data_loader
    
    def evaluate(self, model: torch.nn.Module, iterations: int = None, use_cuda: bool = False) -> float:
        """
        Evaluate the specified model using the specified number of samples from the validation set.
        :param model: The model to be evaluated.
        :param iterations: The number of batches of the dataset.
        :param use_cuda: If True then use a GPU for inference.
        :return: The accuracy for the sample with the maximum accuracy.
        """

        # Your code goes here

        evaluator = ImageNetEvaluator(self._config['dataset_dir'], image_size=image_net_config.dataset['image_size'],
                                      batch_size=image_net_config.evaluation['batch_size'],
                                      num_workers=image_net_config.evaluation['num_workers'])

        return evaluator.evaluate(model, iterations, use_cuda)
    
    def finetune(self, model: torch.nn.Module):
        """
        Finetunes the model.  The implemtation provided here is just an example,
        provide your own implementation if needed.
        :param model: The model to finetune.
        :return: None
        """

        # Your code goes here instead of the example from below

        trainer = ImageNetTrainer(self._config['dataset_dir'], image_size=image_net_config.dataset['image_size'],
                                  batch_size=image_net_config.train['batch_size'],
                                  num_workers=image_net_config.train['num_workers'])

        trainer.train(model, max_epochs=self._config['epochs'], learning_rate=self._config['learning_rate'],
                      learning_rate_schedule=self._config['learning_rate_schedule'], use_cuda=self._config['use_cuda'])

        torch.save(model, os.path.join(self._config['logdir'], 'finetuned_model.pth'))

## 2. Load the Model, Initialize Data Pipeline, Get Starting Accuracy

The next section will initialize the model and the data pipeline. It is also customary to log the original accuracy of the model on the dataset provided before compressing the model.

In [ ]:
data_pipeline = ImageNetDataPipeline(config)

# Input image shape
image_shape = (1, image_net_config.dataset['image_channels'],
               image_net_config.dataset['image_width'], image_net_config.dataset['image_height'])

model = resnet18(pretrained=True)
if config['use_cuda']:
    if torch.cuda.is_available():
        model.to(torch.device('cuda'))
    else:
        raise Exception("use_cuda is True but cuda is unavailable")

accuracy = data_pipeline.evaluate(model, use_cuda=config['use_cuda'])
print(accuracy)

## 3. Compress with Channel Pruning

The next cells perform the actual compression step. First, parameters related to the compression are specified in the following cell:

1. **target_comp_ratio**: The desired compession ratio using Channel Pruning. This value denotes the desired compression % of the original model. To compress the model to 20% of its original size, use 0.2. This would compress the model by 80%. The pre-specified value that is given is 50%

2. **num_comp_ratio_candidates**: The number of compression ratios used by the API at each layer. Note that the model will test multiple different compression ratios per layer to try to compress less-important layers more, in such a way such that the overall compression ratio is equal to target_comp_ratio. The specified value is 10, which means that for each layer, the API will try the values 0.1, 0.2, ... 1.0 as ratios.

3. **cost_metric**: Determines in what way the model is evaluated - can either be compute (mac), or space (memory).

4. **eval_iterations**: The number of batches of data used to evaluate a model while the model is compressing. It is set to 10 to speed up the compression, rather than using the whole dataset. More details are later in the notebook/elsewhere in the AIMET API documentation

5. **modules_to_ignore**: The layers that should be ignored during compression. The first layer is ignored to preserve the way the input interacts with the model; if there are other layers that should be ignored, add them to the list.

6. **num_reconstruction_samples**: During the last stage of Channel Pruning, the Compression API tries to map the outputs of the pruned model with that of the original model through linear regression, and uses this attempt to change the weights in the pruned layer. The regression is done with this many random samples. This should generally be in the 100s.

In [ ]:
target_comp_ratio = Decimal(0.5)

num_comp_ratio_candidates = 10

cost_metric = CostMetric.mac

num_eval_iterations = 10

modules_to_ignore = [model.conv1]

num_reconstruction_samples = 500

The next cell sets up the other parameters needed to perform the compression.

We first define the actual Channel Pruning Parameters. There are two methods for which you can choose parameters - Auto and Manual. For Auto, the only option is a greedy selection scheme, where the optimal compression ratio is selected for each layer among a set list of candidates to reach the target ratio (which was specified in the previous cell). For Manual, you have to specify the compression ratios for each layer; a general rule of thumb, if one is to use Manual, is to start with the ratios found by Auto Mode and use it as a starting point.


In [ ]:
# Creating a Data Loader
data_loader = ImageNetDataLoader(is_training=True, images_dir=config['dataset_dir'],
                                 image_size=image_net_config.dataset['image_size']).data_loader

# Creating Greedy selection parameters:
greedy_params = GreedySelectionParameters(target_comp_ratio=target_comp_ratio,
                                          num_comp_ratio_candidates=num_comp_ratio_candidates)



# Creating Auto mode Parameters:
cp_mode = ChannelPruningParameters.Mode.auto

auto_params = ChannelPruningParameters.AutoModeParams(greedy_select_params=greedy_params, 
                                                      modules_to_ignore=modules_to_ignore)

# Creating Channel Pruning SVD parameters with Auto Mode:
params = ChannelPruningParameters(data_loader=data_loader,
                                  num_reconstruction_samples=num_reconstruction_samples,
                                  allow_custom_downsample_ops=True,
                                  mode=cp_mode,
                                  params=auto_params)

# Scheme is Channel Pruning:
scheme = CompressionScheme.channel_pruning

Finally, the model is compressed using AIMET's ModelCompressor paired with the parameters specified above. This returns both the new model, which is saved, as well as relevant statistics. Finally, the compressed model is evaluated on the dataset. Note here that the ModelCompressor evaluates the model while compressing using the same evaluate function that is in our data pipeline.

In [ ]:
compressed_model, comp_stats = ModelCompressor.compress_model(model=model,
                                                              eval_callback=data_pipeline.evaluate,
                                                              eval_iterations=num_eval_iterations,
                                                              input_shape=image_shape,
                                                              compress_scheme=scheme,
                                                              cost_metric=cost_metric,
                                                              parameters=params)

torch.save(compressed_model, os.path.join(config['logdir'], 'compressed_model.pth'))

print(comp_stats)

comp_accuracy = data_pipeline.evaluate(model, use_cuda=config['use_cuda'])
print(comp_accuracy)

## 4. Finetuning

After the model is compressed, the model is finetuned, then evaluated and saved.

In [ ]:
data_pipeline.finetune(compressed_model)

finetuned_accuracy = data_pipeline.evaluate(compressed_model, use_cuda=config['use_cuda'])
print(finetuned_accuracy)